In [1]:
#!/usr/bin/env python

"""
Plots 
all magnetic field components and total magnetic field
for multiple EUHFORIA time series along with WIND data.
Anwesha Maharana, 2024
"""

import glob 
import datetime
import os.path
import argparse
from datetime import datetime
#import configparser

from ai import cdas #

import numpy as np 
import scipy as sp
import scipy.constants

import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator, AutoLocator
from matplotlib.dates import DayLocator, HourLocator, DateFormatter
from matplotlib.font_manager import FontProperties
import matplotlib.dates as mdates

import csv
from math import *
import pandas as pd

import os
import sys
print(sys.argv)

['/home/u0141347/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py', '-f', '/home/u0141347/.local/share/jupyter/runtime/kernel-756239b4-f87a-4111-8b6a-4bf415ea8ff2.json']


In [22]:
#conditions
plot_obs = np.bool(True) #If you want WIND observations to be plotted along with simulation results
save = np.bool(True) #If you want to save png

/tmp/ipykernel_19494/822415513.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  plot_obs = np.bool(True)
/tmp/ipykernel_19494/822415513.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  save = np.bool(True)


In [14]:
event='20240507'

loc = 'Earth'

pos_label = 'Earth' #'r = 0.4 au, lon = 45$^0$'
pos_save = 'Earth' #'r0p4_lon45'

# time interval you want to plot
date_beg =  "2024-05-04T00:00:00" 
date_end =  "2024-05-18T00:00:00"

start_date= datetime.strptime(date_beg, "%Y-%m-%dT%H:%M:%S")   
end_date = datetime.strptime(date_end, "%Y-%m-%dT%H:%M:%S")

/tmp/ipykernel_19494/2435464823.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  plot_obs = np.bool(True)


In [15]:
#EUHFORIA runs
# directory where data are stored
data_dir = '/home/u0141347/kul/EUHFORIA/runs/'+event+'/' #location where the output will be stored

if not os.path.exists(data_dir):
    os.makedirs(data_dir)
output_dir = data_dir

t7 = '/media/u0141347/T7/EUHFORIA/'#location where EUHFORIA run outputs are stored
run1='20240507_wind1'; run2='20240507_wind2' 
dsv = [t7+run1,t7+run2]
lbl = [run1,run2]
infile = ['hsphere_'+loc, 'hsphere_'+loc, 'hsphere_'+loc]

In [17]:
mu_0 = 4*np.pi*(10**(-7.))  # vacuum magnetic permeability [SI: N / A^2]
mu = 1              # mean molecular weight of the plasma
m_p = 1.6726 * (10**(-27.))     # proton mass [kg]
kB = scipy.constants.k

In [18]:
def interp_nan(A):
    ok = ~np.isnan(A)
    xp = ok.ravel().nonzero()[0]
    fp = A[~np.isnan(A)]
    x  = np.isnan(A).ravel().nonzero()[0]
    A[np.isnan(A)] = np.interp(x, xp, fp)
    return A

# Download WIND data

In [61]:
dd_start=datetime.strptime(str(start_date), "%Y-%m-%d %H:%M:%S")
dd_end=datetime.strptime(str(end_date), "%Y-%m-%d %H:%M:%S")

if plot_obs:
    # get magnetic field data from CDAS
    data_mag = cdas.get_data("sp_phys", "WI_H0_MFI", dd_start, dd_end, ["BF1", "BGSE"], cdf=True) #datetime(2012,9,27), datetime(2012,10,7)
    # filter the magnetic field data
    m = np.logical_and(data_mag["Epoch"] >= dd_start, data_mag["Epoch"] <= dd_end) 
    d = data_mag["Epoch"][m]
    
    b_tot = data_mag["BGSE"][m]
    b_tot[b_tot == -1e31] = np.nan
    b_tot=interp_nan(b_tot)
    
    bx_wind = b_tot[:, 0]; by_wind = b_tot[:,1]; bz_wind = b_tot[:,2];
    b_tot = np.sqrt(bx_wind**2+by_wind**2+bz_wind**2)
    print ('bx_wind',len(bx_wind))
    
    # get plasma data from CDAS
    data_pla = cdas.get_data("sp_phys", "WI_K0_SWE", start_date, end_date, ["V_GSE","Np"], cdf=True)
    
    # filter plasma speed data
    data_pla["V_GSE"][np.any(data_pla["V_GSE"] == -1e31, axis=1), :] = np.nan 
    
    filter_n = data_pla["Np"][0]    
    for i in range(len(data_pla["Np"])):
        if data_pla["Np"][i] == filter_n: data_pla["Np"][i] = np.nan
    
    m = np.logical_and(data_pla["Epoch"] >= start_date, data_pla["Epoch"] <= end_date) 
    dpla = data_pla["Epoch"][m]
    
    v_wind = np.sqrt(data_pla["V_GSE"][:, 0] ** 2 + data_pla["V_GSE"][:, 1] ** 2 + data_pla["V_GSE"][:, 2] ** 2)
    v_wind = interp_nan(v_wind)
    np_wind = interp_nan(data_pla["Np"])

100% [........................................................] 489730 / 489730
bx_wind 20160
100% [........................................................] 310491 / 310491
[-9.9999998e+30  4.3238492e+00  4.0984378e+00 ...  3.3193719e+00
 -9.9999998e+30  3.2629592e+00]
[      nan 4.323849  4.098438  ... 3.319372        nan 3.2629592]
fig created


<Figure size 1120x1120 with 0 Axes>

# Extract EUHFORIA data

In [63]:
def extract_euhforia(sim_loc,infile):
    # directory where data are stored

    file_spr = sim_loc + '/heliosphere/' + infile + ".dsv"

    # column format in euhforia data file 
    data_earth_spr = np.loadtxt(file_spr, delimiter=' ', skiprows=1, usecols=[1,2,3,4,5,6,7,8,9,10,11])

    # dates
    date_earth1 = np.loadtxt(file_spr, delimiter=' ', dtype='str', skiprows=1, usecols=[0])

    # assigning date to euhforia data 
    final_dates1 = []
    for dates in date_earth1:
        final_dates1.append( datetime.strptime(str(dates), "%Y-%m-%dT%H:%M:%S") )

    # ================================================================================================
    # DEFINING VARIABLES
    # ================================================================================================
    
    #defining V and B 
    euhforia_earth_v_spr = np.sqrt(data_earth_spr[:,5]**2 + data_earth_spr[:,6]**2 + data_earth_spr[:,7]**2)
    euhforia_earth_b_spr = np.sqrt(data_earth_spr[:,8]**2 + data_earth_spr[:,9]**2 + data_earth_spr[:,10]**2)

    euhforia_earth_vx_spr = -data_earth_spr[:,5] # = - v_r
    euhforia_earth_vy_spr = -data_earth_spr[:,7] # = - v_lon
    euhforia_earth_vz_spr = -data_earth_spr[:,6] # = - v_clt

    euhforia_earth_bx_spr = -data_earth_spr[:,8] # = - b_r
    euhforia_earth_by_spr = -data_earth_spr[:,10]# = - b_lon
    euhforia_earth_bz_spr = -data_earth_spr[:,9] # = - b_clt

    #defining pressures
    euhforia_earth_p_therm_spr = data_earth_spr[:,4] #[in the order of 10^{-12} Pa]
    euhforia_earth_p_mag_spr = ((euhforia_earth_b_spr*10**-9.0)**2.0)/(2.0*scipy.constants.mu_0)

    euhforia_earth_beta_spr = euhforia_earth_p_therm_spr/euhforia_earth_p_mag_spr #Way to check negative pressure in the domain.... Alert alert alert.

    #defining number density
    euhforia_earth_n_spr = 0.5*data_earth_spr[:,3]

    euhforia_earth_t_spr = euhforia_earth_p_therm_spr/(scipy.constants.k*euhforia_earth_n_spr*1e6)
    euhforia_earth_beta_omni = (4.16*10**-5*euhforia_earth_t_spr + 5.34) * euhforia_earth_n_spr/euhforia_earth_b_spr**2

    return final_dates1, euhforia_earth_vx_spr, euhforia_earth_vy_spr, euhforia_earth_vz_spr, euhforia_earth_bx_spr, euhforia_earth_by_spr, euhforia_earth_bz_spr, euhforia_earth_n_spr, euhforia_earth_p_therm_spr, euhforia_earth_p_mag_spr, euhforia_earth_beta_spr  


# Plotting begins

In [75]:
#Formatting figure
color = ['b-','r-','m-','g-','m-','c-','r','k--','b--','g--','m--']
alpha = [1.0,0.8,1.0,1.0,0.5,0.5]
lw = [1.2,1.2,1.2,1.2,1.2,1.2]
ytic_size = 14
xtic_size = 14
ylab_font = 20
xlab_font = 20

png_name=event+'_'+loc+'_'+'wind'

In [78]:
fig = plt.figure(figsize=(24, 18), dpi=80)
print ('fig created')

num=0; n_cols=6
for i in dsv:
    
    sim_name=i

    final_dates1, euhforia_vx, euhforia_vy, euhforia_vz, euhforia_bx, euhforia_by, euhforia_bz, \
    euhforia_n, euhforia_p_therm, euhforia_p_mag, euhforia_beta = extract_euhforia(sim_name,'hsphere_'+loc)

    # =================================================================================================
    # MAKING PLOTSinfile
    # =================================================================================================  
   
    print("Making nice plots...")
    
    label1 = lbl[num] #'SC_'+str(i+1)
    font = FontProperties()
    font.set_family('serif')
    font.set_name('Times New Roman')
    
    
    # --------------------------------
    # V 
    # --------------------------------

    axis = plt.subplot(n_cols,1,1)

    # axes range
    #axis.set_ylim((400, 800))
    axis.set_xlim((start_date, end_date))
    if num==0: 
        if plot_obs: axis.plot(dpla, v_wind, 'k-', linewidth=0.8)#, label='WIND data')
               
    axis.plot(final_dates1, np.sqrt(euhforia_vx**2+euhforia_vy**2+euhforia_vz**2), color[num], linewidth=lw[num],  label=label1, alpha=alpha[num])
    axis.xaxis.set_minor_locator(AutoMinorLocator(6))    
    plt.setp(axis.get_xticklabels(), visible=False)
    plt.setp(axis.get_yticklabels(), visible=True, fontsize=ytic_size)


    # Show grid
    plt.grid()

    axis.yaxis.set_label_text("$v [km/s]$", fontsize=ylab_font, fontweight='bold')
    #fig.subplots_adjust(left=left)
    plt.legend(bbox_to_anchor=(1.0, 1.0),loc='upper right',prop={'size':16})

    
    # --------------------------------
    # n 
    # --------------------------------

    axis = plt.subplot(n_cols,1,2)

    # axes range
    #axis.set_ylim((ylim_low_v, ylim_up_v))
    #axis.set_ylim((0.1, 100))
    axis.set_xlim((start_date, end_date))
    
    axis.plot(final_dates1, euhforia_n, color[num], linewidth=lw[num],  label=label1, alpha=alpha[num])
    axis.xaxis.set_minor_locator(AutoMinorLocator(6))
    
    if num==0: 
        if plot_obs: axis.plot(dpla, np_wind, 'k-', linewidth=0.8)
    
    plt.setp(axis.get_xticklabels(), visible=False)
    plt.setp(axis.get_yticklabels(), visible=True, fontsize=ytic_size)
    #plt.yscale('log')
    minor_locator = AutoMinorLocator(2)
    #major_locator = AutoLocator()
    axis.yaxis.set_minor_locator(minor_locator)

    # Show grid
    plt.grid()

    axis.yaxis.set_label_text("$n _p[/cm^3]$", fontsize=ylab_font, fontweight='bold')    
    
    # --------------------------------
    # B_x
    # --------------------------------

    axis = plt.subplot(n_cols,1,3)

    # axes range
    #axis.set_ylim((-20, 20))
    axis.set_xlim((start_date, end_date))
    
    if num==0: 
        if plot_obs: axis.plot(d, bx_wind, 'k-', linewidth=0.8)

    axis.plot(final_dates1, euhforia_bx, color[num], linewidth=lw[num],  label=label1, alpha=alpha[num])
    axis.xaxis.set_minor_locator(AutoMinorLocator(6))
    #axis.plot( [final_dates1[-1], final_dates1[-1]], [0, 10], '-k', lw=2, alpha=0.5)
    plt.setp(axis.get_xticklabels(), visible=False)
    plt.setp(axis.get_yticklabels(), visible=True, fontsize=ytic_size)

    # Show grid
    plt.grid()
    axis.yaxis.set_label_text("$B_x [nT]$", fontsize=ylab_font, fontweight='bold')


    # --------------------------------
    # B_y
    # --------------------------------

    axis = plt.subplot(n_cols,1,4)

    # axes range
    #axis.set_ylim((-20, 20))
    axis.set_xlim((start_date, end_date))
    
    if num==0: 
        if plot_obs: axis.plot(d, by_wind, 'k-', linewidth=0.8)

    axis.plot(final_dates1, euhforia_by, color[num], linewidth=lw[num],  label=label1, alpha=alpha[num])
    axis.xaxis.set_minor_locator(AutoMinorLocator(6))
    #axis.plot( [final_dates1[-1], final_dates1[-1]], [0, 10], '-k', lw=2, alpha=0.5)
    plt.setp(axis.get_xticklabels(), visible=False)
    plt.setp(axis.get_yticklabels(), visible=True, fontsize=ytic_size)

    # Show grid
    plt.grid()

    axis.yaxis.set_label_text("$B_y [nT]$", fontsize=ylab_font, fontweight='bold')

    # --------------------------------
    # B_z
    # --------------------------------

    axis = plt.subplot(n_cols,1,5)

    # axes range
    #axis.set_ylim((-20, 20))
    axis.set_xlim((start_date, end_date))

    if num==0: 
        if plot_obs: axis.plot(d, bz_wind, 'k-', linewidth=0.8)
        
    axis.plot(final_dates1, euhforia_bz, color[num], linewidth=lw[num],  label=label1, alpha=alpha[num])
    axis.xaxis.set_minor_locator(AutoMinorLocator(6))
    #axis.plot( [final_dates1[-1], final_dates1[-1]], [0, 10], '-k', lw=2, alpha=0.5)
    plt.setp(axis.get_xticklabels(), visible=False)
    plt.setp(axis.get_yticklabels(), visible=True, fontsize=ytic_size)

    # Show grid
    plt.grid()

    axis.yaxis.set_label_text("$B_z [nT]$", fontsize=ylab_font, fontweight='bold')
    #fig.subplots_adjust(left=left)

    # --------------------------------
    # B_total
    # --------------------------------

    axis = plt.subplot(n_cols,1,6)

    # axes range
    #axis.set_ylim((0, 20))
    axis.set_xlim((start_date, end_date))
    if num==0: 
        if plot_obs: axis.plot(d, b_tot, 'k-', linewidth=0.8)
        
    axis.plot(final_dates1, np.sqrt(euhforia_bx**2+euhforia_by**2+euhforia_bz**2), color[num], linewidth=lw[num],  label=label1, alpha=alpha[num])
    axis.xaxis.set_minor_locator(AutoMinorLocator(12))    
    plt.setp(axis.get_xticklabels(), visible=True)
    plt.setp(axis.get_yticklabels(), visible=True, fontsize=ytic_size)

    # Show grid
    plt.grid()
    #Show legend
    #plt.legend(prop={'size':8})
    axis.yaxis.set_label_text("$|B| [nT]$", fontsize=ylab_font, fontweight='bold')
    
    num = num+1
    

#----------- Overall plot formatting -----------#
#myFmt = mdates.DateFormatter('%d-%m %H:%M') #%d-%m-%y')#
#axis.xaxis.set_major_formatter(myFmt)


font = FontProperties()
font.set_family('serif')
font.set_name('Times New Roman')
# x axis format
myFmt = mdates.DateFormatter('%Y-%m-%d') #%d-%m-%y')#
axis.xaxis.set_major_formatter(myFmt)
#rotating labels
fig.autofmt_xdate() 
plt.xticks(rotation=45)

#ax.xaxis.set_major_locator( MultipleLocator(20) )
axis.xaxis.set_minor_locator(AutoMinorLocator(6)) #( MultipleLocator(10) )

axis.set_xlim((start_date, end_date))
plt.setp(axis.get_xticklabels(), visible=True, fontsize=xtic_size)
axis.xaxis.set_label_text("$Date$", fontsize=xlab_font, fontweight='bold')
fig.align_ylabels()
    
if save: plt.savefig(output_dir + event+'_wind.pdf')

fig created
Making nice plots...
Making nice plots...
